In [6]:
# Step 1: Preprocess the SPY data to create a clean CSV file for Backtrader
import pandas as pd
import os

def preprocess_spy_data():
    """
    Read the original SPY data and create a clean CSV file for Backtrader
    """
    # Read the original CSV file
    spy_file = r'../03_quant_fundamentals/spy_data.csv'
    
    # Read the file skipping the first 3 header rows
    df = pd.read_csv(spy_file, skiprows=3, header=None)
    
    # Set proper column names
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Sort by date (should already be sorted)
    df = df.sort_values('Date')
    
    # Save the clean data
    clean_file = 'spy_data_clean.csv'
    df.to_csv(clean_file, index=False)
    
    print(f"Clean data saved to: {clean_file}")
    print(f"Data shape: {df.shape}")
    print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
    print("\nFirst few rows:")
    print(df.head())
    
    return clean_file

# Create the clean data file
print("=== STEP 1: DATA PREPROCESSING ===")
clean_data_file = preprocess_spy_data()
print("✓ Data preprocessing completed!")

=== STEP 1: DATA PREPROCESSING ===
Clean data saved to: spy_data_clean.csv
Data shape: (1423, 6)
Date range: 2020-01-02 00:00:00 to 2025-08-29 00:00:00

First few rows:
        Date        Open        High         Low       Close    Volume
0 2020-01-02  298.180692  299.424883  297.249847  299.406433  59151200
1 2020-01-03  295.987167  298.272792  295.931872  297.139191  77709700
2 2020-01-06  295.369735  298.355800  295.249920  298.272858  55653900
3 2020-01-07  297.701484  298.180744  296.982622  297.434235  40496400
4 2020-01-08  297.627664  300.245059  297.378837  299.019318  68296000
✓ Data preprocessing completed!


In [ ]:
"""
Step 2: Simple Backtrader Strategy Backtest Example

Indicators and Conditions used only as a reference, this is not a tradable strategy.

STRATEGY TYPE: RSI_MA_Combined
DIRECTION: BUY
INDICATORS: [RSI(28), RSI(21), SMA(100), SMA(50), SMA(200)]

ENTRY CONDITIONS:
  close < SMA(100) and RSI(28) < 30

EXIT CONDITIONS:
  RSI(21) < 25 and SMA(50) < SMA(200)

DETAILED CONDITIONS:
  Entry Logic: close < SMA(100) and RSI(28) < 30
  Exit Logic: RSI(21) < 25 and SMA(50) < SMA(200)
"""

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import backtrader as bt
import backtrader.feeds as btfeeds


class RSIMAStrategy(bt.Strategy):

    params = (
        ('rsi_entry_period', 28),  # Period RSI for entry
        ('rsi_exit_period', 21),   # Period RSI for exit
        ('sma_entry_period', 100), # Period SMA for entry
        ('sma_mid_period', 50),    # Period SMA medium for exit
        ('sma_long_period', 200),  # Period SMA long for exit
        ('rsi_entry_threshold', 30), # Soglia RSI for entry
        ('rsi_exit_threshold', 25),  # Soglia RSI for exit
        ('printlog', False),       # Disabled logging to speed up execution
    )

    def __init__(self):
        # Entry technical indicators
        self.rsi_entry = bt.talib.RSI(self.data.close, timeperiod=self.p.rsi_entry_period)
        self.sma_entry = bt.talib.SMA(self.data.close, timeperiod=self.p.sma_entry_period)

        # Exit technical indicators
        self.rsi_exit = bt.talib.RSI(self.data.close, timeperiod=self.p.rsi_exit_period)
        self.sma_mid = bt.talib.SMA(self.data.close, timeperiod=self.p.sma_mid_period)
        self.sma_long = bt.talib.SMA(self.data.close, timeperiod=self.p.sma_long_period)

        # Tracking variables
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def log(self, txt, dt=None, doprint=False):
        """Logging function for this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    f'BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}')

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log(
                    f'SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')

    def next(self):
        # Log reduced for debug
        if len(self) % 100 == 0:  # Log every 100 days
            self.log(f'Day {len(self)}: Close: {self.data.close[0]:.2f}, '
                    f'RSI28: {self.rsi_entry[0]:.2f}, SMA100: {self.sma_entry[0]:.2f}')

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # ENTRY CONDITIONS: close < SMA(100) and RSI(28) < 30
            entry_sma_condition = self.data.close[0] < self.sma_entry[0]
            entry_rsi_condition = self.rsi_entry[0] < self.p.rsi_entry_threshold

            # Combined entry condition
            if entry_sma_condition and entry_rsi_condition:
                self.log(f'BUY CREATE - Entry conditions met: '
                        f'Close: {self.data.close[0]:.2f} < SMA100: {self.sma_entry[0]:.2f}, '
                        f'RSI28: {self.rsi_entry[0]:.2f} < {self.p.rsi_entry_threshold}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:
            # EXIT CONDITIONS: RSI(21) < 25 and SMA(50) < SMA(200)
            exit_rsi_condition = self.rsi_exit[0] < self.p.rsi_exit_threshold
            exit_sma_condition = self.sma_mid[0] < self.sma_long[0]

            if exit_rsi_condition and exit_sma_condition:
                self.log(f'SELL CREATE - Exit conditions met: '
                        f'RSI21: {self.rsi_exit[0]:.2f} < {self.p.rsi_exit_threshold}, '
                        f'SMA50: {self.sma_mid[0]:.2f} < SMA200: {self.sma_long[0]:.2f}')
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.order = self.sell()

    def stop(self):
        self.log(f'Final Portfolio Value: {self.broker.getvalue():.2f}', doprint=True)


def run_backtest():
    """
    Main function to run the backtest\
    """
    # Initializate Cerebro
    cerebro = bt.Cerebro()

    # Load the data
    data = btfeeds.GenericCSVData(
        # Using the clean SPY data file
        dataname='spy_data_clean.csv',
        dtformat=('%Y-%m-%d'),  # Date format without time
        datetime=0,  # Date column is the first column (index 0)
        open=1,      # Open column is the second column (index 1)
        high=2,      # High column is the third column (index 2)
        low=3,       # Low column is the fourth column (index 3)
        close=4,     # Close column is the fifth column (index 4)
        volume=5,    # Volume column is the sixth column (index 5)
        openinterest=-1,  # No open interest data
        headers=True,  # Skip the header row
        # Date range - use the full range of the data
        fromdate=datetime.datetime(2020, 1, 1),
        todate=datetime.datetime(2025, 8, 31),
    )

    # Add the data to Cerebro
    cerebro.adddata(data)

    # Add the strategy
    cerebro.addstrategy(RSIMAStrategy)

    # Set the initial capital
    cerebro.broker.setcash(100000.0)

    # Set the commission - 0.1% per trade
    cerebro.broker.setcommission(commission=0.001)

    # Add analyzers for performance
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")

    # Add additional analyzers as requested
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name="annual_returns")
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name="time_drawdown")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="detailed_returns")

    # Print the initial portfolio value
    print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')

    # Run the backtest
    results = cerebro.run()

    # Print the final portfolio value
    print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')

    # Extract and print the statistics
    strat = results[0]

    print('\n=== TRADE ANALYSIS ===')
    trade_analysis = strat.analyzers.trades.get_analysis()
    if trade_analysis:
        # Safe handling of trade analysis data using try/except
        try:
            total_trades = trade_analysis.total.total
        except (KeyError, AttributeError):
            total_trades = 0
        print(f"Total Trades: {total_trades}")

        # Safe handling for winning trades
        won_trades = 0
        gross_profit = 0.0
        try:
            won_trades = trade_analysis.won.total
            try:
                gross_profit = trade_analysis.won.pnl.total
            except (KeyError, AttributeError):
                gross_profit = 0.0
        except (KeyError, AttributeError):
            pass
        print(f"Winning Trades: {won_trades}")

        # Safe handling for losing trades
        lost_trades = 0
        gross_loss = 0.0
        try:
            lost_trades = trade_analysis.lost.total
            try:
                gross_loss = trade_analysis.lost.pnl.total
            except (KeyError, AttributeError):
                gross_loss = 0.0
        except (KeyError, AttributeError):
            pass
        print(f"Losing Trades: {lost_trades}")

        print(f"Gross Profit: {gross_profit:.2f}")
        print(f"Gross Loss: {gross_loss:.2f}")

        # Calculate win rate if there are trades
        if total_trades > 0:
            win_rate = (won_trades / total_trades) * 100
            print(f"Win Rate: {win_rate:.2f}%")

        # Profit Factor
        if abs(gross_loss) > 0:
            profit_factor = abs(gross_profit / gross_loss)
            print(f"Profit Factor: {profit_factor:.2f}")

        # # Avg Profit for Trade
        # if total_trades > 0:
        #     avg_profit_per_trade = (gross_profit + gross_loss) / total_trades
        #     print(f"Avg Profit for Trade: {avg_profit_per_trade:.2f}")

        # Net Profit
        net_profit = gross_profit + gross_loss  # gross_loss è già negativo
        print(f"Net Profit: {net_profit:.2f}")
    else:
        print("No trade data available")

    print('\n=== PERFORMANCE METRICS ===')
    sharpe_ratio = strat.analyzers.sharpe.get_analysis()
    if sharpe_ratio and 'sharperatio' in sharpe_ratio:
        print(f"Sharpe Ratio: {sharpe_ratio['sharperatio']:.4f}")

    drawdown = strat.analyzers.drawdown.get_analysis()
    if drawdown:
        print(f"Max Drawdown: {drawdown['max']['drawdown']:.2f}%")
        print(f"Max Drawdown Money: ${drawdown['max']['moneydown']:.2f}")

    returns = strat.analyzers.returns.get_analysis()
    if returns:
        print(f"Total Return: {returns['rtot']*100:.2f}%")
        print(f"Average Return: {returns['ravg']*100:.4f}%")

    # Display results from additional analyzers
    print('\n=== ADDITIONAL ANALYTICS ===')

    # Annual Returns Analysis
    annual_returns = strat.analyzers.annual_returns.get_analysis()
    if annual_returns:
        print("Annual Returns:")
        for year, return_val in annual_returns.items():
            print(f"  {year}: {return_val*100:.2f}%")

    # Time DrawDown Analysis
    time_drawdown = strat.analyzers.time_drawdown.get_analysis()
    if time_drawdown:
        print(f"\nTime DrawDown Analysis:")
        if 'maxdrawdownperiod' in time_drawdown:
            print(f"  Max DrawDown Period: {time_drawdown['maxdrawdownperiod']} days")
        if 'maxdrawdown' in time_drawdown:
            print(f"  Max DrawDown Value: {time_drawdown['maxdrawdown']:.2f}%")

    # Detailed Returns Analysis
    detailed_returns = strat.analyzers.detailed_returns.get_analysis()
    if detailed_returns:
        print(f"\nDetailed Returns Analysis:")
        print(f"  Total Compound Return (rtot): {detailed_returns.get('rtot', 0)*100:.4f}%")
        print(f"  Average Return (ravg): {detailed_returns.get('ravg', 0)*100:.6f}%")
        if 'rnorm' in detailed_returns:
            print(f"  Annualized/Normalized Return (rnorm): {detailed_returns['rnorm']*100:.4f}%")
        if 'rnorm100' in detailed_returns:
            print(f"  Annualized Return in 100% (rnorm100): {detailed_returns['rnorm100']:.4f}%")

    return cerebro, results


print("=== STEP 2: STRATEGY AND BACKTEST FUNCTION DEFINED ===")
print("✓ RSI+MA Strategy class created")
print("✓ Backtest function ready")
print("✓ Ready to run backtest!")


=== STEP 2: STRATEGY AND BACKTEST FUNCTION DEFINED ===
✓ RSI+MA Strategy class created
✓ Backtest function ready
✓ Ready to run backtest!


In [8]:
# Step 3: Execute the backtest
print("=== STEP 3: RUNNING BACKTEST ===")
cerebro, results = run_backtest()

=== STEP 3: RUNNING BACKTEST ===
Starting Portfolio Value: 100000.00
2025-08-29, Final Portfolio Value: 100152.57
Final Portfolio Value: 100152.57

=== TRADE ANALYSIS ===
Total Trades: 1
Winning Trades: 0
Losing Trades: 0
Gross Profit: 0.00
Gross Loss: 0.00
Win Rate: 0.00%
Net Profit: 0.00

=== PERFORMANCE METRICS ===
Sharpe Ratio: -17.1397
Max Drawdown: 0.03%
Max Drawdown Money: $34.64
Total Return: 0.15%
Average Return: 0.0001%

=== ADDITIONAL ANALYTICS ===
Annual Returns:
  2020: 0.00%
  2021: 0.00%
  2022: 0.00%
  2023: 0.00%
  2024: 0.00%
  2025: 0.15%

Time DrawDown Analysis:
  Max DrawDown Period: 10 days
  Max DrawDown Value: 0.03%

Detailed Returns Analysis:
  Total Compound Return (rtot): 0.1525%
  Average Return (ravg): 0.000107%
  Annualized/Normalized Return (rnorm): 0.0270%
  Annualized Return in 100% (rnorm100): 0.0270%
2025-08-29, Final Portfolio Value: 100152.57
Final Portfolio Value: 100152.57

=== TRADE ANALYSIS ===
Total Trades: 1
Winning Trades: 0
Losing Trades: 0
